In [1]:
import glob
import os
import torch
from PIL import Image
from torchvision import transforms
from tqdm import tqdm
from models.generators import CycleGANGenerator as Generator

In [2]:
def tensor_to_pil(tensor):
    to_pil = transforms.ToPILImage()
    image = tensor.to("cpu").clone().detach()
    image = image.squeeze()
    return to_pil(image)

In [3]:
source = glob.glob("../dataset/geinou_256/all/*.png")
len(source)

43067

In [4]:
t = transforms.Compose([transforms.Grayscale(num_output_channels=3),transforms.ToTensor()
                       ])

In [5]:
!ls

 dataloaders   losses		    '<Plug>_'		  train_pix2pix.ipynb
 datasets      models		     test_pix2pix.ipynb
 logs	       network_check.ipynb   trainers


In [6]:
model = Generator(use_dropout=True)
model.load('face2sketch_percept_0.5', 'latest', './logs/face2sketch_percept_0.5')
model.cuda()
model.eval()

CycleGANGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): LeakyReLU(negative_slope=0.01)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Dropout(p=0.2, inplace=False)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): LeakyReLU(negative_slope=0.01)
    (10): ResNetBlock(
      (model): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
        (2): LeakyReLU(negative_slope=0.01)
        (3): ReflectionPad2d((1, 1, 1, 1))
        (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
        (5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affi

In [13]:
for s in tqdm(source):
    image = Image.open(s)
    input = t(image)
    output = model(input[None, :, :].cuda())
    result = tensor_to_pil(output)
    result.save(s.replace('geinou_256/all/','geinou_256/sketch/'))

100%|██████████| 43067/43067 [1:10:35<00:00, 10.17it/s]
